### 0. Prepare expression data from TCGA

In [15]:
import pandas as pd
import glob
import os
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

def addFile(file): 
	df = pd.read_csv(file, sep = "\t")
	name = os.path.basename(file)
	df['file_name'] = name
	df.columns = ['gene', 'fpkm', 'file_name']
	return(df)

def loadExpression(filepath):
	files = glob.glob(filepath)     
	dfs = (addFile(f) for f in files)
	return pd.concat(dfs, ignore_index=True)

def adjust(x): 
	return x+.0001


def topKFeatures(X, y, ifPlot = False, topK = 500):

    bestfeatures = SelectKBest(score_func=chi2, k = topK)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print("print 10 best features:\n", featureScores.nlargest(10,'Score'))
    
    if ifPlot == True:
        plt.figure()
        featureScores.nlargest(10,'Score').plot(kind='barh')
        plt.show()
    
    topk_features = featureScores.nlargest(topK,'Score').Specs.values

    return topk_features

clinical = pd.read_csv("/nfs/home/jaclyns/tcga/nationwidechildrens.org_clinical_patient_prad.txt", sep="\t")
# print(labels)

mapping = pd.read_csv("/nfs/home/jaclyns/tcga/prad.txt", sep="\t")
mapping.columns = ["case_id", "aliquot_id", "file_name"]
# print(mapping)

expression = loadExpression("/nfs/home/jaclyns/tcga/prad/*FPKM-UQ.txt")
# print(expression)

labels = clinical[['bcr_patient_uuid', 'gleason_pattern_primary', 'gleason_pattern_secondary', 'gleason_score']].merge(mapping, right_on='case_id', left_on='bcr_patient_uuid')
labels = labels[['aliquot_id', 'gleason_pattern_primary', 'gleason_pattern_secondary', 'gleason_score']]

expression = expression.merge(mapping, right_on='file_name', left_on='file_name')
expression = expression.dropna(subset=['aliquot_id'])
matrix = expression.pivot(index = 'aliquot_id', columns = 'gene', values = 'fpkm')
fmatrix = matrix.join(labels.set_index('aliquot_id'), on='aliquot_id')

preX = fmatrix.drop('gleason_score', 1)
prey0 = fmatrix[['gleason_pattern_primary']] 
prey1 = fmatrix[['gleason_pattern_secondary']]
prey2 = fmatrix[['gleason_score']]

topfs0 = set(topKFeatures(preX,prey0, topK=1000).tolist())
topfs1 = set(topKFeatures(preX,prey1, topK=1000).tolist())
topfs2 = set(topKFeatures(preX,prey2, topK=1000).tolist())

inter = topfs0.union(topfs1, topfs2)
print(len(inter))
# topfs = topfs0 + topfs1 + topfs2
# topfs = set(topfs)
# print(len(topfs))
features = preX[inter]



print 10 best features:
                     Specs         Score
48631   ENSG00000262902.1  2.486474e+09
5496   ENSG00000124233.11  1.778296e+09
35533   ENSG00000237973.1  1.412967e+09
21558   ENSG00000210082.2  8.031701e+08
5465    ENSG00000124157.6  4.735443e+08
19433   ENSG00000202198.1  4.664365e+08
27198   ENSG00000225972.1  4.464057e+08
48938   ENSG00000263639.4  4.292310e+08
21594   ENSG00000211459.2  3.270682e+08
17968   ENSG00000198886.2  3.180589e+08
print 10 best features:
                     Specs         Score
5496   ENSG00000124233.11  1.601779e+09
48938   ENSG00000263639.4  1.141518e+09
19433   ENSG00000202198.1  6.762889e+08
5465    ENSG00000124157.6  4.421978e+08
21594   ENSG00000211459.2  4.186632e+08
2051   ENSG00000096006.10  3.328794e+08
8446   ENSG00000143632.13  2.990593e+08
48631   ENSG00000262902.1  2.658394e+08
13348   ENSG00000172551.9  2.640845e+08
29500   ENSG00000229314.5  2.442860e+08
print 10 best features:
                     Specs         Score
19433

# Visualization with REFINED

In this notebook we show how to convert tabular data into images, and show those images for visualization. As it is explained in the main markdown, REFINED has two steps: Initialization with manifold learning techniques (MDS), and optimization with a search technique (hill climbing). Therefore to perform visualization with REFINED, we need to do the two steps and once we get the coordinate in the square image for each feature of the tabular data we can generate images associate with each datapoint (sample).

### 1. In the below cell we perform initialization with MDS

In [16]:
import pickle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import scipy.misc
import Toolbox
from Toolbox import two_d_eq, Assign_features_to_pixels
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import euclidean_distances
import math
import os
os.chdir('/nfs/home/jaclyns/REFINED/data')
#%% Loading the data
# empty rows I manually removed from this file
# Feat_DF = pd.read_csv("normalized_padel_feats_NCI60_672_small.csv")
#Feat_DF = pd.read_csv("C:\\Users\\obazgir\\Desktop\\CMDS_IMAGES_NEW\\normalized_padel_feats_NCI60_672.csv")
Feat_DF = features#.head(10)

X = Feat_DF.values; #X = X[:,2:]
original_input = pd.DataFrame(data = X)                              # The MDS input should be in a dataframe format with rows as samples and columns as features
feature_names_list = original_input.columns.tolist()                 # Extracting feature_names_list (gene_names or descriptor_names)
print(">>>> Data  is loaded")

#%% MDS
nn = math.ceil(np.sqrt(len(feature_names_list))) 				     # Image dimension
Nn = original_input.shape[1] 										 # Number of features
    
transposed_input = original_input.T 							     # The MDS input data must be transposed , because we want summarize each feature by two values (as compard to regular dimensionality reduction each sample will be described by two values)
Euc_Dist = euclidean_distances(transposed_input) 					 # Euclidean distance
Euc_Dist = np.maximum(Euc_Dist, Euc_Dist.transpose())   			 # Making the Euclidean distance matrix symmetric

embedding = MDS(n_components=2)										 # Reduce the dimensionality by MDS into 2 components
mds_xy = embedding.fit_transform(transposed_input)					 # Apply MDS			

print(">>>> MDS dimensionality reduction is done")

eq_xy = two_d_eq(mds_xy,Nn)
Img = Assign_features_to_pixels(eq_xy,nn,verbose=1)					# Img is the none-overlapping coordinates generated by MDS

#%% To be saved for hill climbing
Desc = Feat_DF.columns.tolist();  #Desc = Desc[2:]					# Drug descriptors name
Dist = pd.DataFrame(data = Euc_Dist, columns = Desc, index = Desc)	# Generating a distance matrix which includes the Euclidean distance between each and every descriptor
data = (Desc, Dist, Img	)  											# Preparing the hill climbing inputs

# picklename = "Init_MDS_Euc.pickle"
picklename = "Init_PRAD_FPKM_3labels.pickle"
with open(picklename, 'wb') as f:					# The hill climbing input is a pickle, therefore everything is saved as a pickle to be loaded by the hill climbing
    pickle.dump(data, f)

Using TensorFlow backend.


>>>> MDS
>>>> Data  is loaded
>>>> MDS dimensionality reduction is done
>> Assign features to pixels: 762 / 1608
>> Assign features to pixels: 951 / 1608
>> Assign features to pixels: 1028 / 1608
>> Assign features to pixels: 1094 / 1608
>> Assign features to pixels: 1140 / 1608
>> Assign features to pixels: 1182 / 1608
>> Assign features to pixels: 1218 / 1608
>> Assign features to pixels: 1254 / 1608
>> Assign features to pixels: 1282 / 1608
>> Assign features to pixels: 1310 / 1608
>> Assign features to pixels: 1333 / 1608
>> Assign features to pixels: 1357 / 1608
>> Assign features to pixels: 1379 / 1608
>> Assign features to pixels: 1407 / 1608
>> Assign features to pixels: 1423 / 1608
>> Assign features to pixels: 1439 / 1608
>> Assign features to pixels: 1452 / 1608
>> Assign features to pixels: 1462 / 1608
>> Assign features to pixels: 1473 / 1608
>> Assign features to pixels: 1480 / 1608
>> Assign features to pixels: 1486 / 1608
>> Assign features to pixels: 1494 / 1608
>> Ass

### 2. Hill Climbing
Once the initializaiton is performed, then we have to apply the search optimizaition (hill climbing). The below bash script will run the search optimization.

In [ ]:
mpirun -np 32 python3 /nfs/home/jaclyns/REFINED/mpiHill_UF.py --init '/nfs/home/jaclyns/REFINED/data/Init_PRAD_FPKM_3labels.pickle'   --mapping 'Mapping_REFINED_3labels.pickle'  --evolution "REFINED_Evolve_3labels.csv" --num 5


### 3. Visualization
Once the search optimization is performed and the coordinates for features are obtained, we can use the coordinates to generate images for visualization purpose or training CNNs. In the below code visualization is performed.

In [26]:
#%% Visualizing REFFINED images
import math
import pickle
from Toolbox import REFINED_Im_Gen
# MDS
with open('/nfs/home/jaclyns/glow/Mapping_REFINED.pickle','rb') as file:
    gene_names_MDS,coords_MDS,map_in_int_MDS = pickle.load(file)

# # We pick the first 100 data points because of computational costs, but you can pick the entire datasets.
# # X_REFINED_MDS = REFINED_Im_Gen(X[:100,:],nn, map_in_int_MDS, gene_names_MDS,coords_MDS)
X_REFINED_MDS = REFINED_Im_Gen(X,nn, map_in_int_MDS, gene_names_MDS,coords_MDS)

# Font = 20
samples = Feat_DF.index

# aliquot = samples[1]
# print(aliquot)
# X_REFINED_MDS[0,:]

# fig=plt.figure(figsize=(12,8), dpi= 100)
# plt.imshow(X_REFINED_MDS[1,:].reshape(nn,nn), cmap = 'gray') # cmap = 'viridis')
# plt.axis('off')
# plt.savefig('/nfs/home/jaclyns/tcga/'+samples[1]+'.png')
# plt.close(fig)

for i in range(len(samples)):
    fig=plt.figure(figsize=(12,8), dpi= 100)
    plt.imshow(X_REFINED_MDS[i,:].reshape(nn,nn), cmap = 'gray') #cmap = 'viridis')
#     plt.title("Example1", fontsize = Font)
    plt.axis('off')
    fig.savefig('/nfs/home/jaclyns/tcga/refined_images/'+samples[i]+'.png')
    plt.close(fig)

# plt.subplot(142)
# plt.imshow(X_REFINED_MDS[2,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example2", fontsize = Font)
# plt.axis('off')

# plt.subplot(143)
# plt.imshow(X_REFINED_MDS[3,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example3", fontsize = Font)
# plt.axis('off')


# plt.subplot(144)
# plt.imshow(X_REFINED_MDS[4,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example4", fontsize = Font)
# plt.axis('off')

### Converting to 16bit png
Required for Clara Train classification

In [31]:
import json

labels0 = prey0['gleason_pattern_primary'].tolist()
labels1 = prey1['gleason_pattern_secondary'].tolist()
labels2 = prey2['gleason_score'].tolist()

dls = set(labels0)
print(dls)
dls1 = set(labels1)
print(dls1)
dls2 = set(labels2)
print(dls2)

samples = features.index

labelMap0 = {2: 0, 3: 1, 4: 2, 5: 3}
labelMap1 = {3: 0, 4: 1, 5: 2}
labelMap2 = {6: 0, 7: 1, 8: 2, 9: 3, 10: 4}

labelDist0 = {2: 0, 3: 0, 4: 0, 5: 0}
labelDist1 = {3: 0, 4: 0, 5: 0}
labelDist2 = {6: 0, 7: 0, 8: 0, 9: 0, 10: 0}

for s in range(len(samples)):
    labelDist0[labels0[s]] += 1
    labelDist1[labels1[s]] += 1
    labelDist2[labels2[s]] += 1

print(labelDist0)
print(labelDist1)
print(labelDist2)



## we can do binary classification with 2/3 = 0 and 4/5 = 1
## we can split 70/30 training and testing
labels = {}
labelMap = {2: 0, 3: 0, 4: 1, 5: 1}
for s in range(len(samples)):
    labels[samples[s]] = labelMap[labels0[s]]

zeros = [k for k, v in labels.items() if v == 0]
ones = [k for k, v in labels.items() if v == 1]

trainingZeros = zeros[:35]
testingZeros = zeros[35:]
trainingOnes = ones[:35]
testingOnes = ones[35:]

# print(trainingZeros)
print(len(trainingZeros))
# print(trainingZeros)
print(len(testingZeros))

# print(trainingOnes)
print(len(trainingOnes))
# print(testingOnes)
print(len(testingOnes))

training = []
path = "train_primary"
dataset = "/nfs/home/jaclyns/tcga/dataset_primary.json"
for s in trainingZeros + trainingOnes:
    iname = path+"/"+s+".png"
    training.append({ "image": iname, "label": labels[s] })
    
init_json = { "label_format": [1], "data_format": "channels_last", "training": training }
print(json.dumps(init_json, indent=2))

with open(dataset, 'w') as outfile:
    json.dump(init_json, outfile, indent=2)

{2, 3, 4, 5}
{3, 4, 5}
{6, 7, 8, 9, 10}
{2: 2, 3: 49, 4: 49, 5: 1}
{3: 40, 4: 53, 5: 8}
{6: 11, 7: 71, 8: 11, 9: 7, 10: 1}
35
16
35
15
{
  "label_format": [
    1
  ],
  "data_format": "channels_last",
  "training": [
    {
      "image": "train_primary/011453b6-9736-4ea4-8845-bbe4416783d3.png",
      "label": 0
    },
    {
      "image": "train_primary/0c4495e4-b826-45ee-9ad3-873266b4585b.png",
      "label": 0
    },
    {
      "image": "train_primary/0cc5f629-5046-4d05-8a5f-923ce5c04b9e.png",
      "label": 0
    },
    {
      "image": "train_primary/104e14d3-f1d5-443f-88fd-6d67c98b1eff.png",
      "label": 0
    },
    {
      "image": "train_primary/12e19460-da9a-4ff4-8664-801e7f8829b7.png",
      "label": 0
    },
    {
      "image": "train_primary/14830a14-5495-4425-b6ef-78ccffafe03c.png",
      "label": 0
    },
    {
      "image": "train_primary/1c809dd1-f790-458d-9722-c3262afcd416.png",
      "label": 0
    },
    {
      "image": "train_primary/23e9edc6-a5ab-4dca-9243-a